## 11.7 Foody

Aplikacja będzie podobna do poprzedniej, tym razem wykorzystamy `api` https://www.themealdb.com/. Przygotujemy trzy fragmenty
- `FoodListFragment` - fragment wyświetlający listę wszystkich potraw - przefiltrujemy po kraju
- `FoodDetailFragment` - fragment wyświetlający dokładniejsze informacje o danej potrawie
- `FavoriteFragment` - fragment wyświtlający listęv ulubionych potraw

Wykorzystamy `BottomNavigation`, na którym umieścimy fragmenty listy potraw oraz listę ulubionych. Na fragment wyświetlający dokładniejsze infoprmacje będziemy mogli się przenieść z obu fragmentów - przekazany zostanie `id` potrawy. Do stworzenia listy ulubionych potraw wykorzystamy lokalną bazę `ROOM`.

W tej aplikacji posłużymy się pojedynczym `ViewModel` powiązanym z aktywnością hostującą.

<table><tr><td><img src="https://media0.giphy.com/media/lpzoI5Wm7JrPHlXdpd/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/5Zd75RkXZeDJNb46bF/giphy.gif" width="200" /></td></tr></table>

Dodajmy odpowiednie zależności

In [ ]:
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.2"
    }
}

In [ ]:
plugins {
    id 'com.android.application'
    id 'androidx.navigation.safeargs'
}
...
dependencies {

    // ROOM
    implementation "androidx.room:room-runtime:2.4.3"
    annotationProcessor "androidx.room:room-compiler:2.4.3"

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

    // Navigation
    implementation "androidx.navigation:navigation-fragment:2.5.2"
    implementation "androidx.navigation:navigation-ui:2.5.2"

    // Retrofit
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

    // OkHttp
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'

    // Glide
    implementation 'com.github.bumptech.glide:glide:4.14.1'
    annotationProcessor 'com.github.bumptech.glide:compiler:4.14.1'
    ...
}

Dodajmy również odpowiednie upoważnienie do `AndroidManifest`

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

### **`Navigation`**

Rozpocznijmy od utworzenia fragmentów i dodania nawigacji

<img src="https://fv9-4.failiem.lv/down.php?i=bu3zkrzvs" width="400" />

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/mealListFragment">

    <fragment
        android:id="@+id/mealListFragment"
        android:name="pl.udu.uwr.pum.foodyjava.ui.fragments.MealListFragment"
        android:label="fragment_meal_list"
        tools:layout="@layout/fragment_meal_list" >
        <action
            android:id="@+id/action_mealListFragment_to_mealDetailFragment"
            app:destination="@id/mealDetailFragment" >
            <argument
                android:name="id"
                app:argType="string" />
        </action>
    </fragment>
    <fragment
        android:id="@+id/mealDetailFragment"
        android:name="pl.udu.uwr.pum.foodyjava.ui.fragments.MealDetailFragment"
        android:label="fragment_meal_detail"
        tools:layout="@layout/fragment_meal_detail" >
        <action
            android:id="@+id/action_mealDetailFragment_to_mealListFragment"
            app:destination="@id/mealListFragment" />
        <action
            android:id="@+id/action_mealDetailFragment_to_favoriteFragment"
            app:destination="@id/favoriteFragment" />
    </fragment>
    <fragment
        android:id="@+id/favoriteFragment"
        android:name="pl.udu.uwr.pum.foodyjava.ui.fragments.FavoriteFragment"
        android:label="fragment_favorite"
        tools:layout="@layout/fragment_favorite" >
        <action
            android:id="@+id/action_favoriteFragment_to_mealDetailFragment"
            app:destination="@id/mealDetailFragment" >
            <argument
                android:name="id"
                app:argType="string" />
        </action>
    </fragment>
</navigation>

Następnie dodajmy `menu` dla `BottomNavigation`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/foodListFragment"
        android:icon="@drawable/ic_food"
        android:title="@string/food" />
    <item
        android:id="@id/favoriteFragment"
        android:icon="@drawable/ic_favorite"
        android:title="@string/favorite" />
</menu>

Dodajmy nawigację w klasie `MainActivity`

In [ ]:
public class MainActivity extends AppCompatActivity {

    private NavController navController;
    private AppBarConfiguration appBarConfiguration;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        ActivityMainBinding binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        Set<Integer> topLevelDestinationIds = new HashSet<>();
        topLevelDestinationIds.add(R.id.mealListFragment);
        topLevelDestinationIds.add(R.id.favoriteFragment);

        NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.nav_host_fragment);

        if (navHostFragment != null) {
            navController = NavHostFragment.findNavController(navHostFragment);
            NavigationUI.setupWithNavController(binding.bottomNavView, navController);
        }

        appBarConfiguration =  new AppBarConfiguration.Builder(topLevelDestinationIds)
                .build();

        NavigationUI.setupActionBarWithNavController(this, navController, appBarConfiguration);
    }

    @Override
    public boolean onSupportNavigateUp() {
        return NavigationUI.navigateUp(navController, appBarConfiguration) 
            || super.onSupportNavigateUp();
    }
}

Zwróćmy uwagę na `appBarConfiguration`, teraz mamy dwa fragmenty tzw. top level, z których chcemy przejść na inny fragment. Jako argument `AppBarCon figuration` nie podajemy `navController.getGraph()` jak dotychczas, tylko zbiór wszystkich fragmentów które chcemy dodać do `BottomNavigation`

In [ ]:
Set<Integer> topLevelDestinationIds = new HashSet<>();
topLevelDestinationIds.add(R.id.mealListFragment);
topLevelDestinationIds.add(R.id.favoriteFragment);

appBarConfiguration =  new AppBarConfiguration.Builder(topLevelDestinationIds)
        .build();

### **`Retrofit`**

Zmapujmy dane za pomocą strony https://json2csharp.com/code-converters/json-to-pojo, w efekcie dostaniemy kilkadziesiąt pól w klasie `Meal` - wybierzmy kilka

In [ ]:
public class MealResponse{
    public ArrayList<Meal> meals;
}

public class Meal {
    public String idMeal;
    public String strMeal;
    public String strCategory;
    public String strArea;
    public String strInstructions;
    public String strMealThumb;
    public String strTags;
    public String strYoutube;
}

Dodajmy interfejs `MealApi` z dwiema metodami
- `getMealFromApi` - zwraca listę wszystkich potraw, jako `area` wybieramy Polskę
- `getMealById` - zwraca potrawę o zadanych `id`

In [ ]:
public interface MealApi {
    @GET("api/json/v1/1/filter.php?a=Polish")
    Call<MealResponse> getMealFromApi();

    @GET("api/json/v1/1/lookup.php?")
    Call<MealResponse> getMealById(@Query("i")String id);
}

Następnie utwórzmy instancję klasy `Retrofit`, również wykorzystamy `LoggingInterceptor`

In [ ]:
public class RetrofitInstance {
    private RetrofitInstance(){}

    private static volatile MealApi api;
    private static final String baseUrl = "https://www.themealdb.com/";

    public static MealApi getApi() {
        if (api == null) {
            synchronized (RetrofitInstance.class) {
                if (api == null) {
                    HttpLoggingInterceptor interceptor = new HttpLoggingInterceptor();
                    interceptor.setLevel(HttpLoggingInterceptor.Level.BODY);
                    OkHttpClient client = new OkHttpClient.Builder()
                            .addInterceptor(interceptor)
                            .build();
                    api = new Retrofit.Builder()
                            .baseUrl(baseUrl)
                            .addConverterFactory(GsonConverterFactory.create())
                            .client(client)
                            .build().create(MealApi.class);
                }
            }
        }
        return api;
    }
}

### **`ROOM`**

Będziemy chcieli dać użytkownikowi zapisanie ulubionych potraw w bazie danych. Rozpocznijmy od utworzenia `@Entity`, przejdźmy do klasy `Meal` i dodajmy odpowiednią adnotację.

In [ ]:
@Entity(tableName = "meals")
public class Meal {
    @PrimaryKey
    @NonNull
    public String idMeal;
    public String strMeal;
    public String strCategory;
    public String strArea;
    public String strInstructions;
    public String strMealThumb;
    public String strTags;
    public String strYoutube;
}

Następnie utwórzmy interfejs `MealDao` i dodajmy trzy metody pozwalające na wyciuągnięcie wszystkich elementów, dodanie i usunięcie elementu.

In [ ]:
@Dao
public interface MealDao {
    @Insert(onConflict = REPLACE)
    void insert(Meal meal);

    @Delete
    void delete(Meal meal);

    @Query("SELECT * FROM meals")
    LiveData<List<Meal>> getAllMeals();
}

Zdefiniujmy również klasę reprezentującą bazę danych

In [ ]:
@Database(entities = {Meal.class}, version = 1, exportSchema = false)
public abstract class MealDatabase extends RoomDatabase {
    public abstract MealDao mealDao();

    private static volatile MealDatabase INSTANCE;

    private static final int NUMBER_OF_THREADS = 4;
    public static final ExecutorService databaseWriteExecutor =
            Executors.newFixedThreadPool(NUMBER_OF_THREADS);

    public static MealDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (MealDatabase.class) {
                if (INSTANCE == null) {
                    INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                    MealDatabase.class, "meal_database")
                            .build();
                }
            }
        }
        return INSTANCE;
    }
}

### **Szkielet `MVVM`**

Utwórzmy repozytorium z odpowiednimi metodami dostępowymi do naszego `api` oraz `ROOM`.

In [ ]:
public class MealRepository {

    private final MealDao mealDao;

    public MealRepository(Application application) {
        MealDatabase db = MealDatabase.getDatabase(application);
        mealDao = db.mealDao();
    }

    public Call<MealResponse> getMealFromApi(){
        return RetrofitInstance.getApi().getMealFromApi();
    }

    public Call<MealResponse> getMealById(String id){
        return RetrofitInstance.getApi().getMealById(id);
    }

    public void insert(Meal meal){
        mealDao.insert(meal);
    }

    public void delete(Meal meal){
        mealDao.delete(meal);
    }

    public LiveData<List<Meal>> getAllMeals(){
        return mealDao.getAllMeals();
    }
}

Kolejną klasą będzie nazsz `MealViewModel` - w tej aplikacji będziemy posiadać jeden `ViewModel` współdzielony przez wszystkie fragmenty. Ponieważ metoda `getDatabase` wymaga podania `Context`, musimy go przekazać w konstruktorze - wykorzystamy `Application` (najszerszy kontekst).

In [ ]:
public class MealViewModel extends AndroidViewModel {
        public MealViewModel(@NonNull Application application) {
        super(application);
    }
}

Dodajmy repozytorium oraz dwa `MutableLiveData` w których dbierzemy wynik zapytań.

In [ ]:
private final MealRepository repository;

private final MutableLiveData<MealResponse> meals = new MutableLiveData<>();
private final MutableLiveData<MealResponse> meal = new MutableLiveData<>();

Dodajmy gettery

In [ ]:
public LiveData<MealResponse> getMeals() {
    return meals;
}

public LiveData<MealResponse> getMeal() {
    return meal;
}

Ostatnim polem jest `readAllData`, czyli `LiveData` przechowujący odpowiedź z lokalnej bazy

In [ ]:
private LiveData<List<Meal>> favorites;

public LiveData<List<Meal>> getFavorites() {
        return favorites;
    }

W bloku konstruktorze inicjujemy repozytorium

In [ ]:
public MealViewModel(@NonNull Application application) {
    super(application);
    repository = new MealRepository(application);
}

Nastrępnie zdefiniujmy funkcje zwracające listę potraw oraz potrawę po `id`

In [ ]:
public void getMealsFromApi(){
    Call<MealResponse> responseCall = repository.getMealFromApi();

    responseCall.enqueue(new Callback<MealResponse>() {
        @Override
        public void onResponse(@NonNull Call<MealResponse> call, 
                               @NonNull Response<MealResponse> response) {
            if (response.isSuccessful()){
                MealResponse mealResponse = response.body();
                if (mealResponse != null)
                    meals.postValue(mealResponse);
            }
        }

        @Override
        public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
            Log.e(TAG, "error: " + t.getMessage() + "at " + TAG);
        }
    });
}

In [ ]:
public void getMealById(String id){
    Call<MealResponse> responseCall = repository.getMealById(id);

    responseCall.enqueue(new Callback<MealResponse>() {
        @Override
        public void onResponse(@NonNull Call<MealResponse> call, 
                               @NonNull Response<MealResponse> response) {
            if (response.isSuccessful()){
                MealResponse mealResponse = response.body();
                if (mealResponse != null)
                    meal.postValue(mealResponse);
            }
        }

        @Override
        public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
            Log.e(TAG, "error: " + t.getMessage() + "at " + TAG);
        }
    });
}

Ostatnim elementem będą funkcję dodania, usunięcia i odczytu potrawy z lokalnej bazy

In [ ]:
public void getAllMeals(){
    favorites = repository.getAllMeals();
}

public void insert(Meal meal){
    MealDatabase.databaseWriteExecutor.execute(() -> repository.insert(meal));
}

public void delete(Meal meal){
    MealDatabase.databaseWriteExecutor.execute(() -> repository.delete(meal));
}

### **`RecyclerView`**

Wykorzystamy `RecyclerView` dwa razy - dla listy wszystkich potraw, oraz listy ulubionych potraw. Zdefiniujmy więc layout pojedynczego elementu

In [ ]:
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:layout_marginTop="16dp">

    <TextView
        android:id="@+id/name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="nazwa"
        android:textSize="24sp"
        android:layout_margin="8dp"/>

    <ImageView
        android:id="@+id/image"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_height="200dp"
        android:scaleType="centerCrop"
        android:layout_marginStart="25dp"
        android:layout_marginEnd="25dp"
        android:contentDescription="@string/food_image" />

</LinearLayout>

Dodajmy `Comparator`, który (podobnie jak layout) będzie dla obu wspólny

In [ ]:
public class MealComparator extends DiffUtil.ItemCallback<Meal> {
    @Override
    public boolean areItemsTheSame(@NonNull Meal oldItem, @NonNull Meal newItem) {
        return newItem.idMeal.equals(oldItem.idMeal);
    }

    @Override
    public boolean areContentsTheSame(@NonNull Meal oldItem, @NonNull Meal newItem) {
        return newItem.strMeal.equals(oldItem.strMeal);
    }
}

Następnie utwórzmy dwa `ViewHolder`

In [ ]:
public class MealViewHolder extends RecyclerView.ViewHolder {
    private final ListItemRvBinding binding;
    public MealViewHolder(ListItemRvBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(Meal item){
        binding.name.setText(item.strMeal);
        Glide.with(binding.getRoot())
                .load(item.strMealThumb)
                .into(binding.image);
        binding.getRoot().setOnClickListener(view -> {
            NavDirections action = MealListFragmentDirections
                    .actionMealListFragmentToMealDetailFragment(
                            item.idMeal
                    );
            Navigation.findNavController(binding.getRoot()).navigate(action);
        });
    }
}

In [ ]:
public class FavoriteViewHolder extends RecyclerView.ViewHolder {
    private final ListItemRvBinding binding;
    public FavoriteViewHolder(ListItemRvBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(Meal item){
        binding.name.setText(item.strMeal);
        Glide.with(binding.getRoot())
                .load(item.strMealThumb)
                .into(binding.image);
        binding.getRoot().setOnClickListener(view -> {
            NavDirections action = FavoriteFragmentDirections
                    .actionFavoriteFragmentToMealDetailFragment(
                            item.idMeal
                    );
            Navigation.findNavController(binding.getRoot()).navigate(action);
        });
    }
}

Wykorzystujemy `ImageView`, grafikę którą pobieramy wstawiamy za pomocą biblioteki `Glide`. Dodajmy również dwa adaptery.

In [ ]:
public class MealAdapter extends ListAdapter<Meal, MealViewHolder> {
    public MealAdapter(MealComparator comparator) {
        super(comparator);
    }

    @NonNull
    @Override
    public MealViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new MealViewHolder(ListItemRvBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull MealViewHolder holder, int position) {
        Meal item = getItem(position);
        holder.bind(item);
    }
}


In [ ]:
public class FavoriteAdapter extends ListAdapter<Meal, FavoriteViewHolder> {
    public FavoriteAdapter(MealComparator comparator) {
        super(comparator);
    }

    @NonNull
    @Override
    public FavoriteViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new FavoriteViewHolder(ListItemRvBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull FavoriteViewHolder holder, int position) {
        Meal item = getItem(position);
        holder.bind(item);
    }

    public Meal getItemAt(int position){
        return getItem(position);
    }
}

### **`MealListFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/foodRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="36dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Utworzmy `ViewModel`.

In [ ]:
private MealViewModel viewModel;

@Override
public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                         Bundle savedInstanceState) {
    binding = FragmentMealListBinding.inflate(inflater, container, false);
    viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);
    viewModel.getMealsFromApi();
    return binding.getRoot();
}

Dodajmy funkcje pomocnicze

In [ ]:
private void setupRecyclerView(MealAdapter adapter){
    binding.foodRV.setAdapter(adapter);
    binding.foodRV.setLayoutManager(new LinearLayoutManager(requireContext()));
}

W `onViewCreated` wywołuję wcześniej zdefiniowane funkcje i dodajemy obserwator

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    MealAdapter adapter = new MealAdapter(new MealComparator());
    setupRecyclerView(adapter);

    viewModel.getMeals().observe(getViewLifecycleOwner(), 
                                 mealResponse -> adapter.submitList(mealResponse.meals));
}

### **`MealDetailFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        tools:context=".DetailActivity">

        <ProgressBar
            android:id="@+id/progressBar"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignParentStart="true"
            android:layout_alignParentTop="true"
            android:layout_alignParentEnd="true"
            android:layout_gravity="center"
            android:layout_marginStart="181dp"
            android:layout_marginTop="35dp"
            android:layout_marginEnd="182dp"
            android:background="@android:color/transparent"
            android:visibility="invisible" />

        <ImageView
            android:id="@+id/foodImage"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="@string/food_image" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:padding="8dp"
            android:text="@string/title"
            android:theme="@style/ThemeOverlay.AppCompat.Dark" />

        <TextView
            android:id="@+id/category"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/foodImage"
            android:padding="8dp"
            android:text="@string/category"
            android:textColor="?android:textColorSecondary" />

        <TextView
            android:id="@+id/instructions"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/category"
            android:padding="8dp"
            android:text="@string/instructions" />

        <com.google.android.material.floatingactionbutton.FloatingActionButton
            android:id="@+id/favoriteButton"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:layout_alignParentEnd="true"
            android:layout_marginEnd="8dp"
            android:layout_marginTop="16dp"
            android:contentDescription="@string/favorite_button"
            android:src="@drawable/ic_favorite_border" />
    </RelativeLayout>
</ScrollView>

Utwórzmy `ViewModel` i odbierzmy argument

In [ ]:
@Override
public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                         Bundle savedInstanceState) {
    binding = FragmentMealDetailBinding.inflate(inflater, container, false);
    id = requireArguments().getString("id");
    viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);
    viewModel.getMealById(id);
    return binding.getRoot();
}

W metodzie `onCreate` wywołajmy `getMealById` 

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentFoodDetailBinding.inflate(layoutInflater, container, false)
    foodViewModel.getMealById(id!!)
    return binding.root
}

Dodajmy funkcje pomocnicze

In [ ]:
private void inflate(Meal item){
    Glide.with(this)
            .load(item.strMealThumb)
            .into(binding.foodImage);
    binding.category.setText(item.strCategory);
    binding.title.setText(item.strMeal);
    binding.instructions.setText(item.strInstructions);
}

Ponieważ dostaję odpowiedź jako listę, tutaj będzie tob lista jednoelementowa, więc wykorzystuję funkcję `findFirst`

In [ ]:
mealResponse.meals.stream().findFirst().orElse(null))

Następnie wywołuję funkcję `inflate` zdefiniowaną wcześniej

In [ ]:
mealResponse.meals.stream().findFirst().ifPresent(this::inflate)

Dodaję również obsługę `onClick` przycisku dodającego potrawę do ulubionych

In [ ]:
binding.favoriteButton.setOnClickListener(v -> {
    viewModel.insert(mealResponse.meals.stream().findFirst().orElse(null));
});

Pełny kod metody `onViewCreated`

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    viewModel.getMeal().observe(getViewLifecycleOwner(), mealResponse -> {
        mealResponse.meals.stream().findFirst().ifPresent(this::inflate);
        binding.favoriteButton.setOnClickListener(v -> {
            viewModel.insert(mealResponse.meals.stream().findFirst().orElse(null));
        });
    });
}

### **`FavoriteFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/favoriteRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy `ViewModel`

In [ ]:
private MealViewModel viewModel;

@Override
public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                         Bundle savedInstanceState) {
    binding = FragmentFavoriteBinding.inflate(inflater, container, false);
    viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);
    viewModel.getAllMeals();
    return binding.getRoot();
}

Dodajmy dwie funkcje pomocnicze

In [ ]:
private void setupRecyclerView(FavoriteAdapter adapter){
    binding.favoriteRV.setAdapter(adapter);
    binding.favoriteRV.setLayoutManager(new LinearLayoutManager(requireContext()));
}

private void swipeToDelete(FavoriteAdapter adapter) {
    new ItemTouchHelper(new ItemTouchHelper.SimpleCallback(0,
            ItemTouchHelper.RIGHT | ItemTouchHelper.LEFT) {
        @Override
        public boolean onMove(@NonNull RecyclerView recyclerView, 
                              @NonNull RecyclerView.ViewHolder viewHolder, 
                              @NonNull RecyclerView.ViewHolder target) {
            return false;
        }

        @Override
        public void onSwiped(@NonNull RecyclerView.ViewHolder viewHolder, int direction) {
            viewModel.delete(adapter.getItemAt(viewHolder.getAdapterPosition()));
        }
    }).attachToRecyclerView(binding.favoriteRV);
}

W metodzie `onViewCreated` wywołujemy wcześniej zdefiniowane metody

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    FavoriteAdapter adapter = new FavoriteAdapter(new MealComparator());
    setupRecyclerView(adapter);

    viewModel.getFavorites().observe(getViewLifecycleOwner(), adapter::submitList);

    swipeToDelete(adapter);
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media0.giphy.com/media/lpzoI5Wm7JrPHlXdpd/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/5Zd75RkXZeDJNb46bF/giphy.gif" width="150" /></td></tr></table>